<a href="https://colab.research.google.com/github/michael-vdr/Twittorials/blob/master/Quick_win_wordcount_meta_title_analyzer_automated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>





### Resources
Fill in the fields, upload the credentials.json file
This connects with the Search Console API and will scrape meta titles using beautful soup


### Step 0: Language settings / Search console credentials

In [ ]:
langcode="" #@param {type:"string"}
language="" #@param {type:"string"}
consoledomain="" #@param {type:"string"}
brandname="" #@param {type:"string"}


#### Upload credentials.json

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
jsoncredentials = str(uploaded).split("'")[1]
numberqueries=100 #@param


### Step 1 : Import modules

In [ ]:
#import modules
import pandas as pd 
#input/output modules for uploading and downloading files
import datetime
#install nlp modules not yet available in colab
!pip install git+https://github.com/joshcarty/google-searchconsole
!pip install stop_words
!pip install python-levenshtein
import searchconsole
import re #NLP regex module
import nltk #general NLP module
from nltk.stem.snowball import SnowballStemmer #NLP stemming module
from Levenshtein import * #NLP distance caluclator
from stop_words import get_stop_words #stop words NLP library
from bs4 import BeautifulSoup
import requests
nltk.download('punkt') #tokenizer NLP 
nltk.download('stopwords')

### Step 2 : Search queries by page

In [ ]:
#Step 2: upload the page title file
#search console gegevens
from datetime import date
account = searchconsole.authenticate(client_config='client_secrets.json',credentials='credentials.json')
webproperty=account[consoledomain]
datum = date.today().strftime('%Y-%m-%d')
report1 = webproperty.query.range(datum, days=-30).filter('query',brandname,'notContains').dimension('query','page').get()
keywordlist=list()
for row in report1:
    keywordlist.append([row[0],row[1],row[2],row[3],row[4],row[5]])
df = pd.DataFrame(keywordlist)
#columns aanpassen
df.columns=["Search Query","Landing page","Clicks","Impressions","Ctr","Position"]
df['Index']=df.index
df=df.head(numberqueries)
df.head()

### Step 3: Scrape titles function

---



In [ ]:
def get_metatitle(url):
    reqs = requests.get(url)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    title = soup.find("title")
    # displaying the title
    return title.get_text()



### Step 4: Combining dataframes

In [ ]:
df["Title"]=df.apply(lambda x:get_metatitle(x["Landing page"]),axis=1)
df.head()

### Step 5: Count number of words

In [ ]:
stop_words = get_stop_words(langcode)

def number_words(sentence):
  sentencewords = nltk.word_tokenize(str(sentence).lower()) 
  not_stop_words=[]
  for word in sentencewords:
    if(word not in stop_words):
        not_stop_words.append(word)
  sentencelength = len(not_stop_words)
  return sentencelength

df["Words in query"]=df.apply(lambda x:number_words(x["Search Query"]),axis=1)
df["Words in title"]=df.apply(lambda x:number_words(x["Title"]),axis=1)
df.head()

### Step 6: Count number of common words
words of query in title

In [ ]:
def common_words(query,title):
    stemmer  = SnowballStemmer(language, ignore_stopwords=True)
    titlewords = nltk.word_tokenize(str(title).lower())
    stemtitlewords = [stemmer.stem(word) for word in titlewords]
    querywords = nltk.word_tokenize(str(query).lower()) 
    stemquerywords = [stemmer.stem(word) for word in querywords]
    titlenospaces=title.replace(' ','').lower()
    title_only_text = re.sub(r'\W+', '', titlenospaces)
    
    lev_distance=0  
    commonwords=0

    for word in stemquerywords:
      if(word not in stop_words):
          #literal word check
          if(word in title_only_text):
              commonwords=commonwords+1         
          else:
          #check with word with title stemmed and for misspellings
            for word2 in stemtitlewords:
                lev_distance = distance(str(word),str(word2))
                procent_distance = lev_distance/(len(word)+len(word2)/2)
                if procent_distance<0.15:
                    commonwords=commonwords+1
    return commonwords
    
df["Common words"]=df.apply(lambda x:common_words(x["Search Query"],x["Title"]),axis=1)
df.sample(5)

### Step 7 sort and explore table

In [ ]:
%load_ext google.colab.data_table

print("Possible optimizations")
print("----------------------")

df=df.sort_values(by=['Common words', 'Words in query'], ascending=[True, False])
df.head()

### Additional step: Export and download

In [ ]:
#automatically download
df.to_csv('export.csv')
files.download('export.csv') 